In [3]:
%pip install openpyxl

     -------------------------------------- 242.2/242.2 KB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

df = pd.read_excel('../DataForTrainingTasks/DataSet_w_NA.xlsx', sheet_name="Испорченные факты")
df.head()

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3


In [16]:
def prepare_data(x_data, y_data, features):
    dictionary = {
        1: 'Факты.Товар ID', 
        2: 'Маржинальная прибыль_sum',
        3: 'Повторение заказа_sum',
        4: 'Продажи, руб_sum',
        5: 'Продажи, шт_median',
        6: 'Продажи, шт_sum',
        7: 'Доля',
        8: 'sum_d',}
    if dictionary[1] in x_data.columns:
        x_data.drop(dictionary[1], axis=1, inplace=True)
    x_data = x_data[[dictionary[x] for x in features]].values.tolist()
    y_data = y_data.values.tolist()
    return x_data, y_data

def rfc(x_data, y_data, features, values_for_pred, max_depth = 2, test_size = 0.2):
    x_data, y_data = prepare_data(x_data, y_data, features)
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x_data, y_data, test_size = test_size)
    clf = RandomForestClassifier(max_depth=max_depth).fit(X_train, Y_train)
    print("Accuracy: "+str(clf.score(X_test, Y_test)))
    result = clf.predict(values_for_pred if type(values_for_pred[0]) == list else [values_for_pred])
    return result

def gbc(x_data, y_data, features, values_for_pred, max_depth = 2, test_size = 0.2):
    x_data, y_data = prepare_data(x_data, y_data, features)
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x_data, y_data, test_size = test_size)
    clf = GradientBoostingClassifier(max_depth=max_depth).fit(X_train, Y_train)
    print("Accuracy: "+str(clf.score(X_test, Y_test)))
    result = clf.predict(values_for_pred if type(values_for_pred[0]) == list else [values_for_pred])
    return result

def gnbc(x_data, y_data, features, values_for_pred, test_size = 0.2):
    x_data, y_data = prepare_data(x_data, y_data, features)
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x_data, y_data, test_size = test_size)
    clf = GaussianNB().fit(X_train, Y_train)
    print("Accuracy: "+str(clf.score(X_test, Y_test)))
    result = clf.predict(values_for_pred if type(values_for_pred[0]) == list else [values_for_pred])
    return result


df = pd.read_excel('../DataForTrainingTasks/DataSet_w_NA.xlsx', sheet_name="Испорченные факты")
df.info()
df.isnull().any()
df=pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт','Повторение заказа','Маржинальная прибыль','Повторение товара'], index=["Факты.Товар ID"],
                     aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': np.sum,
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum
                        
                               })
newname=df.columns.map('_'.join)
df.columns=newname
df=df.reset_index()
total_sale=df['Продажи, руб_sum'].sum()
df['Доля']=df['Продажи, руб_sum']/total_sale * 100
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)
df=df.assign(sum_d=df['Доля'].cumsum())

df.loc[(df['sum_d'] <= 80), 'ABC'] = 0 #A
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 1 #B
df.loc[(df['sum_d'] > 95), 'ABC'] = 2 #C

x_data = df.loc[:, df.columns != 'ABC']
y_data = df['ABC']

features = [4, 6]
values = [20000.0, 50.0]
print(rfc(x_data, y_data, features, values, test_size=0.1))
print(gbc(x_data, y_data, features, values))
print(gnbc(x_data, y_data, features, values))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10293 entries, 0 to 10292
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Номер заказа            10293 non-null  object 
 1   Возврат                 10293 non-null  object 
 2   Дата заказа             10293 non-null  object 
 3   Дата отправки           10293 non-null  object 
 4   Режим отправки          10293 non-null  object 
 5   Факты.Контрагент ID     10203 non-null  object 
 6   Факты.Товар ID          10242 non-null  object 
 7   Продажи, руб            10098 non-null  float64
 8   Продажи, шт             10272 non-null  float64
 9   Скидка                  10293 non-null  float64
 10  Маржинальная прибыль    10275 non-null  float64
 11  Повторение заказа       10293 non-null  int64  
 12  Повторение контрагента  10293 non-null  int64  
 13  Повторение товара       10293 non-null  int64  
dtypes: float64(4), int64(3), object(7)
mem

C:\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Accuracy: 1.0
[2.]
Accuracy: 0.9757869249394673
[2.]
